In [1]:
# Code Description
'''
Function:
    This script uses Selenium with ChromeDriver to automatically crawl
    product listing data from Taobao search result pages. It is designed
    for e-commerce market analysis and product supply structure research.

Input Parameter:
    KEYWORD  --> Search keyword for products (e.g., lutein, blueberry eye care)

Output Fields:
    'Num'               : Item index (incremented by crawling order)
    'Category'          : Keyword category
    'Detail_Title'      : Product title / description
    'Sold_Count'        : Number of items sold
    'Add_To_Cart'       : Number of users who added the item to cart
    'Repeat_Customers'  : Number of repeat customers
    'Comment_Count'     : Number of product reviews
    'Good_Rate'         : Positive review rate
    'Title_URL'         : URL of the product detail page

Output:
    Structured product listing data (can be further saved as Excel / CSV files)
'''


# Library Imports
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from pyquery import PyQuery as pq     # For parsing and querying HTML content
import time                          # Control page loading and waiting time
import openpyxl as op                # Excel file read/write operations
import re                            # Regular expressions for text cleaning

In [2]:
# Global variable
count = 1     # Row index for writing data into Excel

# Six predefined keyword categories
KEYWORDS = [
    "叶黄素",
    "叶黄素 玉米黄质",
    "蓝莓 护眼",
    "虾青素",
    "DHA",
    "护眼 保健品",
]

# Maximum number of items to crawl per keyword
MAX_ITEMS = 50

# Initialize ChromeDriver options
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ['enable-automation'])

from selenium.webdriver.chrome.service import Service
service = Service(
    executable_path="/Users/xiaowei/desktop/7201/chromedriver-mac-arm64/chromedriver"
)

driver = webdriver.Chrome(options=options, service=service)

# Anti-bot configuration: hide Selenium automation flag
driver.execute_cdp_cmd(
    "Page.addScriptToEvaluateOnNewDocument",
    {"source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"""}
)

# Maximize browser window
driver.maximize_window()

# Open Taobao search homepage
driver.get('https://s.taobao.com/')

# Explicit wait for dynamic page loading
wait = WebDriverWait(driver, 20)

In [3]:
def Crawer_main():
    try:
        for KEYWORD in KEYWORDS:
            print(f"\n========== Start crawling keyword: {KEYWORD} ==========\n")

            search_url = f"https://s.taobao.com/search?q={KEYWORD}"
            driver.get(search_url)
            time.sleep(2)

            # Crawl only the first page and pass the keyword to get_goods as the category
            get_goods(1, KEYWORD)

    except Exception as exc:
        print("Error in Crawer_main function! Error: {}".format(exc))

In [4]:
def search_goods(kw):
    """
    Automatically detects whether the current page is the homepage
    or a search results page, and selects the appropriate search box
    to perform the search.
    """

    time.sleep(1)

    try:
        # Try the search box on the results page (higher priority)
        search_input = driver.find_element(By.CSS_SELECTOR, "#J_Search input#q")
        search_input.clear()
        search_input.send_keys(kw)
        time.sleep(0.2)

        search_button = driver.find_element(By.CSS_SELECTOR, "#J_Search button.btn-search")
        search_button.click()

        print(f"🔍 Search succeeded using the results-page search box: {kw}")
        time.sleep(2)
        return

    except:
        print("⚠️ Not on a results page, trying the homepage search box...")

    try:
        # Homepage search box (used for the first search)
        home_input = wait.until(
            EC.element_to_be_clickable((By.ID, "q"))
        )
        home_input.clear()
        home_input.send_keys(kw)
        time.sleep(0.2)

        home_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
        home_button.click()

        print(f"🏠 Search succeeded using the homepage search box: {kw}")
        time.sleep(2)
        return

    except Exception as e:
        print("❌ Homepage search also failed:", e)

In [5]:
# Jump to pageStart
def turn_pageStart():
    try:
        print(f"Jumping to page {pageStart}")

        # Scroll to the bottom to trigger pagination loading
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # Page number input box
        page_input = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, ".next-pagination-jump-input input")
        ))
        page_input.clear()
        page_input.send_keys(str(pageStart))

        # Click the confirm button
        confirm_btn = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, ".next-pagination-jump-go")
        ))
        confirm_btn.click()

        # Wait until the page number is updated
        wait.until(
            EC.text_to_be_present_in_element(
                (By.CSS_SELECTOR, ".next-pagination-display em"),
                str(pageStart)
            )
        )

        print(f"Successfully jumped to page {pageStart}")
        time.sleep(2)

    except Exception as exc:
        print(f"turn_pageStart error: {exc}")


# Page turning function
def page_turning(page_number):
    try:
        print(f"Turning to page {page_number}")
        time.sleep(2)

        # Locate the "Next Page" button (class name is next-next)
        next_btn = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, ".next-pagination-item.next-next")
        ))
        next_btn.click()

        # Wait until the displayed page number updates
        wait.until(
            EC.text_to_be_present_in_element(
                (By.CSS_SELECTOR, ".next-pagination-display em"),
                str(page_number)
            )
        )

        print(f"Successfully turned to page {page_number}")
        time.sleep(1)

    except Exception as exc:
        print(f"page_turning error: {exc}")

In [6]:
def parse_detail_page(detail_url):
    data = {
        "Detail_Title": None,
        "Repeat_Customers": None,
        "Add_To_Cart": None,
        "Comment_Count": None,
        "Sold_Count": None,
        "Good_Rate": None,
    }

    try:
        main_handle = driver.current_window_handle
        driver.execute_script("window.open(arguments[0]);", detail_url)
        driver.switch_to.window(driver.window_handles[-1])

        try:
            wait.until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "span.mainTitle--R75fTcZL")
                )
            )
        except TimeoutException:
            pass

        time.sleep(1.2)

        html = driver.page_source
        doc = pq(html)

        # Product title on the detail page
        data["Detail_Title"] = doc("span.mainTitle--R75fTcZL").text()

        # Repeat customers / Add to cart
        for span in doc(".itemInfo--TSMo4Asj span").items():
            t = span.text().strip()

            if "回头客" in t:
                data["Repeat_Customers"] = t

            elif ("加购" in t):
                data["Add_To_Cart"] = t.replace(" ", "")

        # Comment count
        # Covers formats such as: 600+ / (4000+) / 1万+
        comment_text = doc("span.tabDetailItemTitle--bJtPXTNu").text()
        if comment_text:
            m = re.search(r'(\d+\.?\d*万?\+?)', comment_text)
            if m:
                data["Comment_Count"] = m.group(1)

        # Positive review rate (e.g., 99.1% / 98.4% in the last 3 months)
        rate_text = doc("span.rateTextTitle--rlYtTd7U").text()
        if rate_text:
            m = re.search(r'(\d+\.?\d*%)', rate_text)
            data["Good_Rate"] = m.group(1) if m else rate_text

        # Sold count (sold / monthly sales / total sold)
        full_text = doc.text()

        m = re.search(r"已售\s*([0-9\.万\+]+)", full_text)
        if m:
            data["Sold_Count"] = m.group(1)
        else:
            m2 = re.search(r"(月售|已卖出)\s*([0-9\.万\+]+)", full_text)
            if m2:
                data["Sold_Count"] = m2.group(2)

        driver.close()
        driver.switch_to.window(main_handle)

    except Exception as exc:
        print(f"parse_detail_page error: {exc}")
        try:
            if len(driver.window_handles) > 1:
                driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except:
            pass

    return data

In [7]:
def get_goods(max_items, category, max_retry=2):
    """
    Automatically crawl products until max_items are collected (default: 50).
    category = current keyword, written into the Category column in Excel.
    max_retry = maximum retry attempts per single product.
    """
    global count, wb

    collected = 0
    page_num = 1

    while collected < max_items:
        print(f"\n--- Parsing keyword [{category}] - Page {page_num} ---")

        try:
            wait.until(
                EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, "a.doubleCardWrapperAdapt--mEcC7olq")
                )
            )
        except:
            print("Page loading failed, stopping this keyword")
            break

        time.sleep(1)

        items = driver.find_elements(By.CSS_SELECTOR, "a.doubleCardWrapperAdapt--mEcC7olq")
        print(f"Found {len(items)} products on this page")

        if len(items) == 0:
            print("No products found on this page, stopping this keyword")
            break

        for idx, item in enumerate(items, start=1):
            if collected >= max_items:
                print(f"\nCollected {max_items} items, keyword completed")
                return

            title_url = item.get_attribute("href")

            success = False
            retry = 0

            while not success and retry <= max_retry:
                try:
                    detail = parse_detail_page(title_url)

                    # Core field fallback check (prevent empty data)
                    if not detail.get("Detail_Title"):
                        raise Exception("Detail_Title is empty")

                    # ===== Write to Excel =====
                    wb.cell(row=count, column=1, value=count - 1)          # Num
                    wb.cell(row=count, column=2, value=category)           # Category
                    wb.cell(row=count, column=3, value=detail["Detail_Title"])
                    wb.cell(row=count, column=4, value=detail["Sold_Count"])
                    wb.cell(row=count, column=5, value=detail["Add_To_Cart"])
                    wb.cell(row=count, column=6, value=detail["Repeat_Customers"])
                    wb.cell(row=count, column=7, value=detail["Comment_Count"])
                    wb.cell(row=count, column=8, value=detail["Good_Rate"])
                    wb.cell(row=count, column=9, value=title_url)

                    print(f"Write succeeded: {category} item {collected+1} → {detail['Detail_Title']}")

                    count += 1
                    collected += 1
                    success = True

                except Exception as e_item:
                    retry += 1
                    print(f"⚠ Retry {retry}/{max_retry}: {e_item}")
                    time.sleep(1)

            if not success:
                print("❌ Product failed multiple times, skipped but will continue to fill quota")

        # Turn to next page
        page_num += 1
        try:
            page_turning(page_num)
        except:
            print("Page turning failed, ending this keyword early")
            break

    print(f"\n== Keyword [{category}] completed, total collected: {collected} ==\n")


In [8]:
if __name__ == '__main__':

    # Create Excel file
    try:
        ws = op.Workbook()              # Entire workbook
        wb = ws.create_sheet(index=0)   # First worksheet

        title_list = [
            'Num',
            'Category',
            'Detail_Title',
            'Sold_Count',
            'Add_To_Cart',
            'Repeat_Customers',
            'Comment_Count',
            'Good_Rate',
            'Title_URL',
        ]

        for i, col in enumerate(title_list, start=1):
            wb.cell(row=count, column=i, value=col)

        count += 1
        
        print("Excel initialized successfully!")

    except Exception as exc:
        print("Failed to create Excel file! Error: {}".format(exc))


    # Main loop: automatically crawl six keyword categories
    MAX_ITEMS = 50   # Maximum of 50 items per keyword

    for kw in KEYWORDS:
        print(f"\n========== Start crawling keyword: {kw} ==========\n")

        search_goods(kw)           # Search for the keyword
        get_goods(MAX_ITEMS, kw)   # Automatically paginate until 50 items are collected


    # Save Excel file
    data = time.strftime('%Y%m%d-%H%M', time.localtime())
    Filename = f"Taobao_{data}.xlsx"
    ws.save(filename=Filename)

    print("\n========== All keywords have been crawled ==========")
    print(Filename + " saved successfully!")


Excel 初始化成功！

========== 开始爬取关键词：叶黄素 ==========

⚠️ 当前不是搜索结果页，尝试使用首页搜索框...
🏠 使用【首页搜索框】搜索成功：叶黄素

--- 解析关键词【叶黄素】第 1 页 ---
本页找到 52 个商品
写入成功：叶黄素 第 1 条 → DLOVE【诺奖得主研发】水溶性叶黄素虾青素亮眼护眼猫去泪痕狗GJ DLOVE【诺奖得主研发】水溶性叶黄素虾青素亮眼护眼猫去泪痕狗GJ
写入成功：叶黄素 第 2 条 → 德国双心高端护眼宝胶囊越橘蓝莓鱼油护眼叶黄素成人护眼丸omega3 德国双心高端护眼宝胶囊越橘蓝莓鱼油护眼叶黄素成人护眼丸omega3
写入成功：叶黄素 第 3 条 → Youthit优思益蓝莓叶黄素护眼丸成人越橘胶囊花青素进口保健品 Youthit优思益蓝莓叶黄素护眼丸成人越橘胶囊花青素进口保健品
写入成功：叶黄素 第 4 条 → Swisse叶黄素成人护眼片保护眼睛保健品官方正品旗舰店蓝莓越橘 Swisse叶黄素成人护眼片保护眼睛保健品官方正品旗舰店蓝莓越橘
写入成功：叶黄素 第 5 条 → 汤臣倍健Yep花青素亮眼蓝莓叶黄素青少年成人呵护眼睛营养咀嚼片 汤臣倍健Yep花青素亮眼蓝莓叶黄素青少年成人呵护眼睛营养咀嚼片
写入成功：叶黄素 第 6 条 → 朴诺叶黄素成人护眼中老年专用保健品蓝莓胶囊朴诺官方正品旗舰店 朴诺叶黄素成人护眼中老年专用保健品蓝莓胶囊朴诺官方正品旗舰店
写入成功：叶黄素 第 7 条 → 叶黄素护眼蓝莓美国进口高浓缩胶囊眼睛保健品成人中老年旗舰店 叶黄素护眼蓝莓美国进口高浓缩胶囊眼睛保健品成人中老年旗舰店
写入成功：叶黄素 第 8 条 → 澳洲Youthit优思益蓝莓叶黄素护眼丸成人胶囊DHA新升级进口2瓶 澳洲Youthit优思益蓝莓叶黄素护眼丸成人胶囊DHA新升级进口2瓶
写入成功：叶黄素 第 9 条 → 朴诺蓝莓素叶黄素成人护眼丸片眼睛保健品官方正品美国进口 朴诺蓝莓素叶黄素成人护眼丸片眼睛保健品官方正品美国进口
写入成功：叶黄素 第 10 条 → 北京同仁堂朕皇官方正品叶黄素酯蓝莓越橘儿童学生成人中老年软糖 北京同仁堂朕皇官方正品叶黄素酯蓝莓越橘儿童学生成人中老年软糖
写入成功：叶黄素 第 11 条 → 德国双心护眼叶黄素成人护眼丸胶囊45粒正品官方旗舰店 德国双心护眼叶黄素成